## Linear Regression with threshold optimization
Need to add crossvalidation

In [1]:
#Import all the files to be shared among all notebooks 
import utils
import preprocessing
import data_visualization
import feature_engineering
from ML_algorithms import *
import pandas as pd
from seaborn import countplot

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = utils.get_dataset()

In [3]:
df = preprocessing.encode_education(df)
df = preprocessing.one_hot_encoding(df,columns = ["Marital_Status"])
df = preprocessing.encode_days_as_costumer(df)
df = feature_engineering.drop_useless_columns(df)

In [4]:
df = preprocessing.impute_income_KNN(df)

In [ ]:
# get a list of all columns
columns = df.columns.tolist()

In [ ]:
# remove target feature Response from the list
columns.remove("Response")

In [ ]:
# very imbalanced dataset
countplot(x="Response", data=df)

In [ ]:
seed = 0 

In [ ]:
# perform the train, test split
X_train, X_test, y_train, y_test = utils.data_split(df, test_size=0.2, random_state=seed)

In [ ]:
X_train.head()

In [ ]:
yX_train = pd.concat([y_train, X_train], axis=1)

In [ ]:
corr = yX_train.corr()

In [ ]:
#Checking independent variables for linear relationship to outcome variable
#To remove variables with weak correlation
corr["Response"]

In [ ]:
#Dropping variables with corr below modulus(0.05)
yX_train1 = yX_train.drop(["Year_Birth", "NumDealsPurchases", "NumStorePurchases", "NumWebVisitsMonth", "Complain", "Marital_Status_Absurd", "Marital_Status_Alone", "Marital_Status_Divorced", "Marital_Status_Widow", "Marital_Status_YOLO"], axis=1)

In [ ]:
corr1 = yX_train1.corr()
corr1["Response"]

In [ ]:
#Dropping variables with corr below modulus(0.1)
yX_train2 = yX_train1.drop(["Education", "Kidhome", "Marital_Status_Together", "Marital_Status_Married", "MntFruits", "MntFishProducts", "MntSweetProducts"], axis=1)

In [ ]:
corr2 = yX_train2.corr()
corr2["Response"]

In [ ]:
corr2
#possibly drop NumCatalogPurchases as it introduces multicollinearity
yX_train3 = yX_train2.drop(["NumCatalogPurchases"], axis=1)

In [ ]:
yX_train3.head()

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

y1, X1 = dmatrices('Response ~ Income + Teenhome + Dt_Customer + Recency + MntWines + MntMeatProducts + MntGoldProds + NumWebPurchases + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5 + AcceptedCmp1 + AcceptedCmp2 + Marital_Status_Single', data = yX_train3, return_type='dataframe')

lm1 = sm.OLS(y1, X1).fit()
print(lm1.summary())

y2, X2 = dmatrices('Response ~ Income + Teenhome + Dt_Customer + Recency + MntWines + MntMeatProducts + NumWebPurchases + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5 + AcceptedCmp1 + AcceptedCmp2 + Marital_Status_Single', data = yX_train3, return_type='dataframe')

lm2 = sm.OLS(y2, X2).fit()
print(lm2.summary())

y3, X3 = dmatrices('Response ~ Teenhome + Dt_Customer + Recency + MntWines + MntMeatProducts + NumWebPurchases + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5 + AcceptedCmp1 + AcceptedCmp2 + Marital_Status_Single', data = yX_train3, return_type='dataframe')

lm3 = sm.OLS(y3, X3).fit()
print(lm3.summary())

y4, X4 = dmatrices('Response ~ Teenhome + Dt_Customer + Recency + MntMeatProducts + NumWebPurchases + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5 + AcceptedCmp1 + AcceptedCmp2 + Marital_Status_Single', data = yX_train3, return_type='dataframe')

lm4 = sm.OLS(y4, X4).fit()
print(lm4.summary())

y5, X5 = dmatrices('Response ~ Teenhome + Dt_Customer + Recency + NumWebPurchases + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5 + AcceptedCmp1 + AcceptedCmp2 + Marital_Status_Single', data = yX_train3, return_type='dataframe')

lm5 = sm.OLS(y5, X5).fit()
print(lm5.summary())

In [ ]:
X_train1 = yX_train[["Teenhome", "Dt_Customer", "Recency", "NumWebPurchases", "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Marital_Status_Single"]]
y_train1 = yX_train[["Response"]]

In [ ]:
X_train1.shape

(1792, 10)

In [26]:
len(y_train1)

1792

In [28]:
X_test.shape

(448, 32)

In [23]:
X_train1.shape

In [ ]:
X_test = X_test[["Teenhome", "Dt_Customer", "Recency", "NumWebPurchases", "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Marital_Status_Single"]]

In [ ]:
X_test.shape

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
model.fit(X_train1, y_train1)
y_predicted = model.predict(X_test)

In [ ]:
threshold = utils.max_threshold(y_predicted, y_test, threshold_range = (0.1, 0.6),iterations=1000, visualization=True)

In [ ]:
y_pred = utils.predict_with_threshold(y_predicted,threshold)
print("Profit:", utils.profit_share(y_pred, y_test))